In [1]:
import numpy as np
import pandas as pd
import csv
from collections import defaultdict
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn import svm
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score 
from sklearn.linear_model import LogisticRegression
import random

In [4]:
#Reading the dataset with sentiment analysis of reviews
merged_wavg = pd.read_csv('merged_wavg.csv')
merged_wavg.head()

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,Bacterial Skin Infection,A + D Cracked Skin Relief,"""I have severe cracked skin on my hands. I&#0...",Positive,10.000000,6
1,Otitis Media,A / B Otic,"""It numbs the pain. It makes my ear feel heavi...",Positive,10.000000,20
2,HIV Infection,Abacavir / dolutegravir / lamivudine,"""I was diagnosed in January 2011. My own immun...",Negative,8.753846,6
3,HIV Infection,Abacavir / dolutegravir / lamivudine,"""I recently found out about my positive status...",Positive,8.753846,26
4,HIV Infection,Abacavir / dolutegravir / lamivudine,"""I was diagnosed in 2007 and had since never h...",Neutral,8.753846,9


In [5]:
#Sorting dataset and grouping by disease
merged_wavg = merged_wavg.sort_values(['Disease','Rating_Wavg'],ascending=False, ignore_index=True).groupby('Disease').head(10060)
merged_wavg

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,zen Shoulde,Voltaren,"""Great help""",Positive,8.854352,33
1,zen Shoulde,Indomethacin,"""It works.""",Neutral,8.757336,12
2,zen Shoulde,Diclofenac,"""While taking this medication I had increased ...",Negative,8.617426,13
3,zen Shoulde,Diclofenac,"""This medication has been a God send for me. ...",Negative,8.617426,11
4,zen Shoulde,Diclofenac,"""Great help""",Positive,8.617426,33
...,...,...,...,...,...,...
161292,NaN,Miconazole,"""So please don&#039;t scratch rub or mess with...",Negative,3.280507,0
161293,NaN,Miconazole,"""Product works but the dispenser is terrible. ...",Negative,3.280507,1
161294,NaN,Multivitamin with minerals,"""Severe hives itching after taking for 6 months""",Negative,3.267327,0
161295,NaN,Gadobutrol,"""Was given for MRI of heart dye to suspected ...",Positive,1.000000,0


In [6]:
# Exporting the dataset with weighted average
merged_wavg = merged_wavg.to_csv("Drug_Recommender.csv", index=False)
merged_wavg = pd.read_csv("Drug_Recommender.csv")
merged_wavg

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,zen Shoulde,Voltaren,"""Great help""",Positive,8.854352,33
1,zen Shoulde,Indomethacin,"""It works.""",Neutral,8.757336,12
2,zen Shoulde,Diclofenac,"""While taking this medication I had increased ...",Negative,8.617426,13
3,zen Shoulde,Diclofenac,"""This medication has been a God send for me. ...",Negative,8.617426,11
4,zen Shoulde,Diclofenac,"""Great help""",Positive,8.617426,33
...,...,...,...,...,...,...
142564,NaN,Miconazole,"""So please don&#039;t scratch rub or mess with...",Negative,3.280507,0
142565,NaN,Miconazole,"""Product works but the dispenser is terrible. ...",Negative,3.280507,1
142566,NaN,Multivitamin with minerals,"""Severe hives itching after taking for 6 months""",Negative,3.267327,0
142567,NaN,Gadobutrol,"""Was given for MRI of heart dye to suspected ...",Positive,1.000000,0


In [7]:
# Taking drugs only with positive reviews for recommendation
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Negative"].index)
merged_wavg = merged_wavg.drop( merged_wavg[merged_wavg['Review_Sentiment'] == "Neutral"].index)

In [8]:
merged_wavg.head()

,Disease,Drug,Review,Review_Sentiment,Rating_Wavg,UsefulCount
0,zen Shoulde,Voltaren,"""Great help""",Positive,8.854352,33
4,zen Shoulde,Diclofenac,"""Great help""",Positive,8.617426,33
5,zen Shoulde,Relafen,"""I&#039;m probably the only one I know taking ...",Positive,8.442509,50
6,zen Shoulde,Ibuprofen,"""I&#039;ve found that taking ibuprofen (200 mg...",Positive,8.428997,0
8,zen Shoulde,Naproxen,"""Very little relief. I finished PT and after ...",Positive,7.947768,6


In [10]:
# Taking predicted disease as input and recommending drug based on highest weighted average and useful count of ratings
groupedByCount = merged_wavg.groupby(['Disease', 'Drug', 'Rating_Wavg'])['UsefulCount'].sum().reset_index()
groupedByCount

,Disease,Drug,Rating_Wavg,UsefulCount
0,0</span> users found this comment helpful.,Aviane,7.550182,0
1,0</span> users found this comment helpful.,Bepreve,6.694915,0
2,0</span> users found this comment helpful.,Cryselle,6.963415,0
3,0</span> users found this comment helpful.,Depo-Provera,4.875840,0
4,0</span> users found this comment helpful.,Drysol,9.385661,0
...,...,...,...,...
6302,zen Shoulde,Ibuprofen,8.428997,0
6303,zen Shoulde,Nabumetone,7.513756,50
6304,zen Shoulde,Naproxen,7.947768,6
6305,zen Shoulde,Relafen,8.442509,50


In [13]:
groupedByDisease = groupedByCount.groupby('Disease')
groupedByDisease.head()

,Disease,Drug,Rating_Wavg,UsefulCount
0,0</span> users found this comment helpful.,Aviane,7.550182,0
1,0</span> users found this comment helpful.,Bepreve,6.694915,0
2,0</span> users found this comment helpful.,Cryselle,6.963415,0
3,0</span> users found this comment helpful.,Depo-Provera,4.875840,0
4,0</span> users found this comment helpful.,Drysol,9.385661,0
...,...,...,...,...
6301,zen Shoulde,Diclofenac,8.617426,33
6302,zen Shoulde,Ibuprofen,8.428997,0
6303,zen Shoulde,Nabumetone,7.513756,50
6304,zen Shoulde,Naproxen,7.947768,6


In [17]:
predicted_disease = ['Rheumatoid Arthritis']
recommended_drug = pd.DataFrame(groupedByDisease.get_group((predicted_disease[0])))
recommended_drug

,Disease,Drug,Rating_Wavg,UsefulCount
5356,Rheumatoid Arthritis,Abatacept,8.234631,571
5357,Rheumatoid Arthritis,Absorbine Jr.,10.000000,1
5358,Rheumatoid Arthritis,Acetaminophen / hydrocodone,8.694243,363
5359,Rheumatoid Arthritis,Acetaminophen / tramadol,8.255422,66
5360,Rheumatoid Arthritis,Actemra,8.917933,167
...,...,...,...,...
5424,Rheumatoid Arthritis,Valdecoxib,9.682927,8
5425,Rheumatoid Arthritis,Vimovo,8.453564,45
5426,Rheumatoid Arthritis,Voltaren,8.854352,172
5427,Rheumatoid Arthritis,Xeljanz,6.902542,266


In [21]:
print("Recommended drugs for this disease are:\n ", recommended_drug["Drug"].unique())

Recommended drugs for this disease are:
  ['Abatacept' 'Absorbine Jr.' 'Acetaminophen / hydrocodone'
 'Acetaminophen / tramadol' 'Actemra' 'Acthar' 'Adalimumab' 'Aleve'
 'Anakinra' 'Arava' 'Arthrotec' 'Aspirin' 'Auranofin' 'Azathioprine'
 'Bextra' 'Celebrex' 'Celecoxib' 'Certolizumab' 'Cimzia' 'Corticotropin'
 'Deltasone' 'Depo-Medrol' 'Diclofenac' 'Diclofenac / misoprostol'
 'Diflunisal' 'Doxycycline' 'Enbrel' 'Esomeprazole / naproxen'
 'Etanercept' 'Etodolac' 'Famotidine / ibuprofen' 'Feldene' 'Golimumab'
 'Humira' 'Hydroxychloroquine' 'Ibuprofen' 'Imuran' 'Infliximab'
 'Kenalog-40' 'Leflunomide' 'Lorcet 10 / 650' 'Lortab' 'Meloxicam'
 'Methotrexate' 'Methyl salicylate' 'Methylprednisolone' 'Minocycline'
 'Mobic' 'Naproxen' 'Orencia' 'Oxaprozin' 'Piroxicam' 'Plaquenil'
 'Prednicot' 'Prednisone' 'Remicade' 'Rheumatrex Dose Pack' 'Ridaura'
 'Rituxan' 'Rituximab' 'Simponi' 'Simponi Aria' 'Sulfasalazine'
 'Tocilizumab' 'Tofacitinib' 'Tramadol' 'Triamcinolone' 'Ultracet'
 'Valdecoxib' 'Vi